# LLM Zoomcamp 2024 Competition
This competition requires the use of LLMs to solve high school mathematics problems. The task is to develop models that can accurately solve the problems and then the predictions submitted.

The model in this run is Mistral.

In [2]:
# Import required modules.
import os
import re
import pandas as pd
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from scorer import score
from mistralai import Mistral

/home/codespace/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Set up Mistral API key.
load_dotenv(Path(find_dotenv()))
api_key = os.getenv("MISTRAL_API_KEY")

In [10]:
# Define the model.
llm = "mistral-large-latest"
client = Mistral(api_key=api_key)

In [25]:
# Function to query the LLM.
def get_answer(question):
    instruction = f'{question} Return as a float'
    chat_response = client.chat.complete(
        model = llm,
        messages = [
            {
                "role": "user",
                "content": instruction,
            },
        ]
    )

    return chat_response.choices[0].message.content

In [17]:
def return_answer(text):
    # Use regular expression to find all numbers in the text
    numbers = re.findall(r'-?\d+\.?\d*', text)

    if numbers:
        # Convert the last found number to a float and return it
        return float(numbers[-1])
    else:
        # Return None if no numbers are found
        return None

In [18]:
# Function to prepare the dataset and get answers.
pool = ThreadPoolExecutor(max_workers=6)

def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [19]:
# Function for processing each row.
def process_row(row):
    problem_id = row['problem_id']
    problem_text = row['problem_text']

    llm_reasoning = get_answer(problem_text)

    numerical_answer = return_answer(llm_reasoning)

    return {
        'problem_id': problem_id,
        'problem_text': problem_text,
        'llm_reasoning': llm_reasoning,
        'answer': numerical_answer
    }

In [3]:
# Load the train data.
df_train = pd.read_csv('data/train.csv')
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [26]:
# Process row.
rows = df_train.head().to_dict(orient='records')
process_row(rows[0])

{'problem_id': 2374,
 'problem_text': 'Find the value of the expression $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$.',
 'llm_reasoning': 'To find the value of the expression \\(\\dfrac{17}{5} : \\dfrac{34}{3} + 1.3\\), we need to follow the order of operations, which is division and multiplication first, followed by addition and subtraction.\n\n1. **Division**:\n   \\[\n   \\dfrac{17}{5} : \\dfrac{34}{3} = \\dfrac{17}{5} \\times \\dfrac{3}{34}\n   \\]\n   Simplify the division by multiplying by the reciprocal:\n   \\[\n   \\dfrac{17}{5} \\times \\dfrac{3}{34} = \\dfrac{17 \\times 3}{5 \\times 34} = \\dfrac{51}{170}\n   \\]\n\n2. **Simplify the fraction**:\n   \\[\n   \\dfrac{51}{170}\n   \\]\n   This fraction can be simplified if needed, but for the calculation, we can keep it as is.\n\n3. **Addition**:\n   \\[\n   \\dfrac{51}{170} + 1.3\n   \\]\n   Convert \\(1.3\\) to a fraction:\n   \\[\n   1.3 = \\dfrac{13}{10}\n   \\]\n\n4. **Common denominator**:\n   To add \\(\\dfrac{51}{170}\\) and \

In [5]:
rows = df_train.head().to_dict(orient='records')
rows[1]

{'problem_id': 4723,
 'problem_text': 'In a company of 30 people, 25 use the social network "Odnoklassniki" and 10 use the social network "VKontakte". Choose the statements that are true under the given conditions.\n\n\\begin{center}\n\\begin{tabularx}{\\textwidth}{p{0.1cm}X}  \n1) & In this company, there will be 10 people who do not use either "Odnoklassniki" or "VKontakte". \\\\ \n2) & In this company, there will be at least 5 people using both networks. \\\\ \n3) & There will not be a single person in this company who uses only "Odnoklassniki". \\\\ \n4) & No more than 10 people in this company use both networks. \\\\ \n\\end{tabularx}\\end{center}\n\nIn the answer, write the numbers of the selected statements without spaces, commas, or other additional symbols.',
 'answer': '24'}

In [28]:
# Return results df_train head.
results = map_progress(pool, rows, process_row)
df_results = pd.DataFrame(results)
df_results

 20%|█████████████████▌                                                                      | 1/5 [00:00<00:00, 10.28it/s]


SDKError: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}

In [ ]:
# Show df_train head.
df_train.head()

In [ ]:
# Score df_train head results.
score(df_train.head(), df_results, 'problem_id')

In [ ]:
# Put all together in a function.
def prepare_prompts_and_get_answers(df):
    rows = df.to_dict(orient='records')
    results = map_progress(pool, rows, process_row)
    return pd.DataFrame(results)

In [ ]:
# Return the results head.
df_train_results = prepare_prompts_and_get_answers(df_train)
df_train_results.head()

In [ ]:
# Get the total score.
score(df_train, df_train_results, 'problem_id')

In [ ]:
# Prepare the test submission.
df_test = pd.read_csv('data/test.csv')

df_test_results = prepare_prompts_and_get_answers(df_test)

submission = df_test_results[['problem_id', 'answer']]
submission.to_csv('submission_mistral', index=False)